Задание 3. Рассчитайте базовые метрики для когорты 8-14.01.2021:
- cost
- installs 
- buyers 3 - плательщики 3-го дня когорты
- buyers
- CPM
- IPM
- CPI
- ARPU_7
- ARPPU_7
- ARPU
- cost per buyer 1,3,5,7,14 - цена за плательщика на указанный день когорты
- ROAS 1,3,5,7,14,30 - фактическая окупаемость на указанный день когорты


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from operator import attrgetter
import matplotlib.pyplot as plt
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [2]:
def date_preprocess(s):
    try:
        return datetime.strptime(s+":00", '%Y-%m-%d %H:%M:%S.%f%z')
    except ValueError:
        return datetime.strptime(s+":00", '%Y-%m-%d %H:%M:%S%z')

In [39]:
def date(s):
    return datetime.strptime(s, '%Y-%m-%d')

In [3]:
user_data = pd.read_csv("data/user_data.csv")
user_data['install_time'] = user_data['install_time'].apply(date_preprocess)
user_data['event_time'] = user_data['event_time'].apply(date_preprocess)
user_data.head()

,attributed_touch_time,install_time,event_time,event_name,event_revenue_usd,media_source,channel,campaign_id,adset_id,ad_id,country_code,user_id
0,NaN,2021-01-01 00:08:10.721000+00:00,2021-01-01 00:08:10.721000+00:00,install,NaN,NaN,NaN,NaN,NaN,NaN,VN,1609459686080-7483030925743023347
1,NaN,2021-01-01 00:39:23.948000+00:00,2021-01-01 00:39:23.948000+00:00,install,NaN,NaN,NaN,NaN,NaN,NaN,US,1609461562630-105981105396801528
2,NaN,2021-01-01 00:40:36.525000+00:00,2021-01-01 00:40:36.525000+00:00,reinstall,NaN,NaN,NaN,NaN,NaN,NaN,CZ,1609461625904-7673362380866907786
3,NaN,2021-01-01 00:49:03.486000+00:00,2021-01-01 00:49:03.486000+00:00,reinstall,NaN,NaN,NaN,NaN,NaN,NaN,PT,1609462209340-904240965619919015
4,NaN,2021-01-01 00:51:53.395000+00:00,2021-01-01 00:51:53.395000+00:00,install,NaN,NaN,NaN,NaN,NaN,NaN,US,1609279214347-3174174


In [4]:
mask = (user_data['install_time'] >= '2021-1-8 00:00:00') & (user_data['install_time'] < '2021-1-15 00:00:00')
user_data = user_data.loc[mask]

колво платящих пользователй

колво покупки новых пользоватлей


In [12]:
installs = user_data['event_name'].loc[user_data['event_name']=='install'].count()

In [16]:
all_users = len(user_data['user_id'].unique())

In [22]:
paying_users = len(user_data['user_id'].loc[(user_data['event_name'] == "af_purchase") | (user_data['event_name'] == "af_subscribe")].unique())


In [51]:
revenue = user_data['event_revenue_usd'].loc[(user_data['event_name'] == "af_purchase") | (user_data['event_name'] == "af_subscribe")].sum()


In [82]:
df_final = user_data[['install_time','event_time','event_name','event_revenue_usd','country_code','user_id']].loc[(user_data['event_name'] == "af_purchase") | (user_data['event_name'] == "af_subscribe")]
df_final.reset_index(drop= True , inplace= True )
df_final['order_day'] = df_final['event_time'].dt.to_period('D')
df_final['type'] = df_final.groupby('user_id')['install_time'].transform('min').dt.to_period('D') 
df_final = df_final.drop(df_final.loc[df_final.order_day  < df_final.type].index)
cohort = df_final.groupby(['type', 'order_day']).agg({'user_id':pd.Series.nunique,'event_revenue_usd':sum}).reset_index(drop=False)
cohort['period'] = cohort.order_day - cohort.type
cohort['period'] = cohort['period'].apply(attrgetter('n'))

In [83]:
cohort

,type,order_day,user_id,event_revenue_usd,period
0,2021-01-08,2021-01-08,509,6286.215977,0
1,2021-01-08,2021-01-09,453,6820.134145,1
2,2021-01-08,2021-01-10,267,5822.962230,2
3,2021-01-08,2021-01-11,294,4642.407567,3
4,2021-01-08,2021-01-12,264,4837.653485,4
...,...,...,...,...,...
443,2021-01-14,2021-03-11,80,2219.354268,56
444,2021-01-14,2021-03-12,213,5119.562546,57
445,2021-01-14,2021-03-13,103,1984.327394,58
446,2021-01-14,2021-03-14,70,1701.147421,59


In [172]:
buyers_3 = cohort['user_id'].loc[cohort.period == 2].sum()

In [41]:
ads_data = pd.read_csv("data/ads_data.csv")
ads_data['date'] = ads_data['date'].apply(date)

In [44]:
ads_mask = (ads_data['date'] >= '2021-1-8 00:00:00') & (ads_data['date'] < '2021-1-15 00:00:00')
ads_data = ads_data.loc[ads_mask]

In [46]:
ads_data.reset_index(drop= True , inplace= True )
ads_data.head()

,date,campaign_id,ad_set_id,ad_id,cpm,ctr,spend,clicks,impressions,breakdown_set,breakdowns,platform
0,2021-01-08,23847157133130369,23847157133120369,23847157133270369,13.316456,0.506329,5.26,2,395,{country},"{""country"": ""UK""}",ios
1,2021-01-08,23847157133170369,23847157133180369,23847157133260369,10.000000,0.647249,3.09,2,309,{country},"{""country"": ""UK""}",android
2,2021-01-08,23847157133130369,23847157133120369,23847157133250369,27.763975,1.863354,4.47,3,161,{country},"{""country"": ""UK""}",ios
3,2021-01-08,23847157133130369,23847157133120369,23847157133240369,23.108696,0.000000,10.63,0,460,{country},"{""country"": ""UK""}",ios
4,2021-01-08,23847157133130369,23847157133120369,23847157133230369,31.917808,1.369863,4.66,2,146,{country},"{""country"": ""UK""}",ios


In [173]:
cost = ads_data['spend'].sum()

In [174]:
impressions = ads_data['impressions'].sum()

In [175]:
CPM =  cost / impressions * 1000

In [176]:
IPM = installs / impressions * 1000

In [177]:
CPI = cost / installs

In [178]:
ARPU = revenue / all_users

In [179]:
ARPPU = revenue / paying_users

In [180]:
cost_per_buyer  = cost / paying_users

In [181]:
spend_by_date = pd.DataFrame(ads_data.groupby('date').agg(spend=('spend','sum')))
spend_by_date['type'] = spend_by_date.index

spend_by_date

,spend,type
date,,
2021-01-08,242091.88,2021-01-08
2021-01-09,256254.46,2021-01-09
2021-01-10,271115.82,2021-01-10
2021-01-11,264614.84,2021-01-11
2021-01-12,286323.70,2021-01-12
2021-01-13,290997.26,2021-01-13
2021-01-14,241885.14,2021-01-14


In [159]:
cohort_pivot = cohort.loc[cohort.period == 11-1]
cohort_pivot['type'] = cohort_pivot['type'].apply(lambda x: x.to_timestamp())

cohort_pivot = pd.merge(cohort_pivot, spend_by_date, on ='type')
    
cohort_pivot

,type,order_day,user_id,event_revenue_usd,period,spend
0,2021-01-08,2021-01-18,160,2389.109956,10,242091.88
1,2021-01-09,2021-01-19,299,4982.370456,10,256254.46
2,2021-01-10,2021-01-20,228,3959.739716,10,271115.82
3,2021-01-11,2021-01-21,122,2485.145039,10,264614.84
4,2021-01-12,2021-01-22,143,2595.065429,10,286323.70
5,2021-01-13,2021-01-23,171,3166.296899,10,290997.26
6,2021-01-14,2021-01-24,137,2353.123893,10,241885.14


In [164]:
def cost_per_buyer(day):
    spend_by_date = pd.DataFrame(ads_data.groupby('date').agg(spend=('spend','sum')))
    spend_by_date['type'] = spend_by_date.index

    cohort_pivot = cohort.loc[cohort.period == day]
    cohort_pivot['type'] = cohort_pivot['type'].apply(lambda x: x.to_timestamp())

    cohort_pivot = pd.merge(cohort_pivot, spend_by_date, on ='type')
    
    cost = cohort_pivot['spend'].loc[cohort_pivot.period == day].sum()
    paying_users = cohort_pivot['user_id'].loc[cohort_pivot.period == day].sum()
    
    return cost / paying_users

In [165]:
cost_per_buyer_10 = cost_per_buyer(10)
cost_per_buyer_10

1470.859603174603

In [169]:
def ROAS(day):
    spend_by_date = pd.DataFrame(ads_data.groupby('date').agg(spend=('spend','sum')))
    spend_by_date['type'] = spend_by_date.index

    cohort_pivot = cohort.loc[cohort.period == day]
    cohort_pivot['type'] = cohort_pivot['type'].apply(lambda x: x.to_timestamp())

    cohort_pivot = pd.merge(cohort_pivot, spend_by_date, on ='type')
    
    cost = cohort_pivot['spend'].loc[cohort_pivot.period == day].sum()
    revenue = cohort_pivot['event_revenue_usd'].loc[cohort_pivot.period == day].sum()
    
    return (revenue / cost)*100

In [171]:
ROAS(10)

1.1833513934271565

In [187]:
final = pd.DataFrame({'metric':['cost','installs','buyers_3','buyers','CPM','IPM','CPI','ARPU','ARPPU','cost_per_buyer_10','ROAS_10'],
             'value':[cost,installs,buyers_3,paying_users,CPM,IPM,CPI,ARPU,ARPPU,cost_per_buyer_10,ROAS(10)]})

final['value'] = final['value'].astype('float').apply(lambda x: '%.3f' % x)


In [188]:
final

,metric,value
0,cost,1853283.100
1,installs,268457.000
2,buyers_3,1756.000
3,buyers,12022.000
4,CPM,11.323
5,IPM,1.640
6,CPI,6.903
7,ARPU,3.459
8,ARPPU,89.463
9,cost_per_buyer_10,1470.860
